In [2]:
import os
import glob
import numpy as np
import tensorflow as tf
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
labelDf = pd.read_csv("train_labels.csv")
labelDf = labelDf.set_index('planet_id')
labelDf

In [ ]:
labelDf = pd.read_csv("train_labels.csv")
labelDf = labelDf.set_index('planet_id')

mean = np.mean(labelDf.mean())
std = np.std(labelDf.std())
max = np.max(labelDf.max())
min = np.min(labelDf.min())
mean, std, max, min

for col in labelDf.columns:
    labelDf.loc[:,col] = (labelDf[col] - mean) / (std)
labelDf

In [ ]:
star = labelDf.index[0]
file_path = 'train/'+str(star)+'/combined.npz'
with np.load(file_path) as data:
        data = data['a'][0,:,0:283,:]

features = np.reshape(data,(-1,25,283,4))
features = np.mean(features,axis=1)
print(features.shape)
dataDf = pd.DataFrame(data[:,:,1], columns=labelDf.columns)
dataDfFiltered = pd.DataFrame(features[:,:,1], columns=labelDf.columns)

In [ ]:
wavelength = labelDf.columns[0]
print(wavelength)

print(labelDf.loc[star,wavelength])
fig = go.Figure()
fig.add_trace(go.Scatter(y=dataDf[wavelength],mode='markers',name='flux',marker=dict(size=3, color='blue'), visible='legendonly'))
#fig.add_trace(go.Scatter(y=dataDf[wavelength].diff(),mode='markers',name='diff',marker=dict(size=3, color='red')))
for i in [100,300,500,1000]:#[2,6,10,20,100]:
    fig.add_trace(go.Scatter(y=dataDf[wavelength].rolling(i, center=True).mean(),mode='markers',name=f'rolling_{i}',marker=dict(size=3)))
    #fig.add_trace(go.Scatter(y=dataDf[wavelength].diff().rolling(i,center=True).mean(),mode='markers',name=f'diff_roll_{i}',marker=dict(size=3)))
fig.show()

fig2 = go.Figure()
fig2.add_trace(go.Scatter(y=dataDfFiltered[wavelength],mode='markers',name='flux',marker=dict(size=3, color='blue'), visible='legendonly'))
#fig.add_trace(go.Scatter(y=dataDf[wavelength].diff(),mode='markers',name='diff',marker=dict(size=3, color='red')))
for i in [10,30,50]:#[2,6,10,20,100]:
    fig2.add_trace(go.Scatter(y=dataDfFiltered[wavelength].rolling(i, center=True).mean(),mode='markers',name=f'rolling_{i}',marker=dict(size=3)))
    #fig.add_trace(go.Scatter(y=dataDf[wavelength].diff().rolling(i,center=True).mean(),mode='markers',name=f'diff_roll_{i}',marker=dict(size=3)))
fig2.show()

In [ ]:
fig2 = go.Figure()
for i,wavelength in enumerate(labelDf.columns):
    if i%10 != 0:
        continue
    #fig2.add_trace(go.Scatter(y=dataDfFiltered[wavelength],mode='markers',name='flux',marker=dict(size=3, color='blue'), visible='legendonly'))
    fig2.add_trace(go.Scatter(y=dataDfFiltered[wavelength].rolling(50, center=True).mean() / dataDfFiltered[wavelength].max(),mode='markers',name=f'rolling_{wavelength}',marker=dict(size=3)))
fig2.show()

In [15]:
est = []
for i,wavelength in enumerate(labelDf.columns):
    roll = dataDfFiltered[wavelength].rolling(50, center=True).mean()# / dataDfFiltered[wavelength].max()
    est.append((roll.max()-roll.min())/roll.max())

In [ ]:
labelDf.loc[star],est

In [ ]:
# 1. filter & get some mean to remove noise
# 2. calculate difference of top / bottom
# 3. difference / top = spectrum


# confirmed the target is delF / F -> delta flux / flux
# normalize by flux -> we have to introduce a scaling such that we can scale the flux to 1 -> target will be absolute then
# lots of cnns to filter for the mean